In [94]:
# !pip install z3-solver

In [95]:
# !pip install utils

In [96]:
from itertools import combinations
from z3 import *
import utils
import math

# Define problem parameters

In [97]:
v = 9
items_size = [5,6,5,3,8,1,1,4]
number_of_items = len(items_size)
couriers_capacity = [15,7,5]
number_of_couriers = len(couriers_capacity)
# This line defines the distance matrix between each location point.
distance = [[0 ,1 ,3 ,4 ,2 ,5 ,6 ,6 ,4 ,6],
            [1 ,0 ,4 ,2 ,5 ,6 ,3 ,3 ,1 ,5],
            [3 ,4 ,0 ,9 ,8 ,9 ,5 ,7 ,1 ,8],
            [4 ,2 ,9 ,0 ,1 ,2 ,3 ,2 ,2 ,5],
            [2 ,5 ,8 ,1 ,0 ,4 ,3 ,7 ,5 ,3],
            [5 ,6 ,9 ,2 ,4 ,0 ,8 ,1 ,8 ,7],
            [6 ,3 ,5 ,3 ,3 ,8 ,0 ,4 ,8 ,9],
            [6 ,3 ,7 ,2 ,7 ,1 ,4 ,0 ,4 ,1],
            [4 ,1 ,1 ,2 ,5 ,8 ,8 ,4 ,0 ,8],
            [6 ,5 ,8 ,5 ,3 ,7 ,9 ,1 ,8 ,0]]
            
# represents whether an item is assigned to a courier
items_assigned = [[Bool(f"p_{i}_{j}") for j in range(number_of_couriers)] for i in range(number_of_items)]
length = [Int(f"l_{i}") for i in range(number_of_couriers)]

s = Solver()

## Each item is delivered by exactly one courier


In [98]:
def at_least_one(bool_vars):
    return Or(bool_vars)

def at_most_one(bool_vars):
    return [Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)]

def exactly_one(bool_vars):
    return at_most_one(bool_vars) + [at_least_one(bool_vars)]

In [99]:
for item in range(number_of_items):
      s.add(exactly_one([items_assigned[item][courier] for courier in range(number_of_couriers)]))
s

[Not(And(p_0_0, p_0_1)),
 Not(And(p_0_0, p_0_2)),
 Not(And(p_0_1, p_0_2)),
 Or(p_0_0, p_0_1, p_0_2),
 Not(And(p_1_0, p_1_1)),
 Not(And(p_1_0, p_1_2)),
 Not(And(p_1_1, p_1_2)),
 Or(p_1_0, p_1_1, p_1_2),
 Not(And(p_2_0, p_2_1)),
 Not(And(p_2_0, p_2_2)),
 Not(And(p_2_1, p_2_2)),
 Or(p_2_0, p_2_1, p_2_2),
 Not(And(p_3_0, p_3_1)),
 Not(And(p_3_0, p_3_2)),
 Not(And(p_3_1, p_3_2)),
 Or(p_3_0, p_3_1, p_3_2),
 Not(And(p_4_0, p_4_1)),
 Not(And(p_4_0, p_4_2)),
 Not(And(p_4_1, p_4_2)),
 Or(p_4_0, p_4_1, p_4_2),
 Not(And(p_5_0, p_5_1)),
 Not(And(p_5_0, p_5_2)),
 Not(And(p_5_1, p_5_2)),
 Or(p_5_0, p_5_1, p_5_2),
 Not(And(p_6_0, p_6_1)),
 Not(And(p_6_0, p_6_2)),
 Not(And(p_6_1, p_6_2)),
 Or(p_6_0, p_6_1, p_6_2),
 Not(And(p_7_0, p_7_1)),
 Not(And(p_7_0, p_7_2)),
 Not(And(p_7_1, p_7_2)),
 Or(p_7_0, p_7_1, p_7_2)]

## Each courier delivers no more items than their capacity.




In [100]:
def at_most_k_np(bool_vars, k):
    return And([Or([Not(x) for x in X]) for X in combinations(bool_vars, k + 1)])

In [101]:
def at_most_k_np_with_sizes(bool_vars, size_vars, k):
   clauses = []
   for i in range(len(bool_vars)):
      for combination in combinations(range(len(bool_vars)), i+1):
         total_size = Sum([size_vars[j] for j in combination])
         if i+1 > k or total_size > k:
            clauses.append(Or([Not(bool_vars[j]) for j in combination]))
   return And(clauses)


In [102]:
for item in range(number_of_items):
   for courier in range(number_of_couriers):
    #  at_most_k_np probleme ici on ne prends pas en compte le poids de chaque item mais l'item en lui même. Exemple si un coursier à une capacité de 10, et que il y a deux item respectivement de poids 5 et 6, ici la clause sera vide car le solver la considerera tlt vrai car il ne prends pas en compte le poids de chaque item et ici 2<10, hors c'est 11 
      s.add(at_most_k_np_with_sizes([items_assigned[item][courier]], [items_size[item]],couriers_capacity[courier]))
s

[Not(And(p_0_0, p_0_1)),
 Not(And(p_0_0, p_0_2)),
 Not(And(p_0_1, p_0_2)),
 Or(p_0_0, p_0_1, p_0_2),
 Not(And(p_1_0, p_1_1)),
 Not(And(p_1_0, p_1_2)),
 Not(And(p_1_1, p_1_2)),
 Or(p_1_0, p_1_1, p_1_2),
 Not(And(p_2_0, p_2_1)),
 Not(And(p_2_0, p_2_2)),
 Not(And(p_2_1, p_2_2)),
 Or(p_2_0, p_2_1, p_2_2),
 Not(And(p_3_0, p_3_1)),
 Not(And(p_3_0, p_3_2)),
 Not(And(p_3_1, p_3_2)),
 Or(p_3_0, p_3_1, p_3_2),
 Not(And(p_4_0, p_4_1)),
 Not(And(p_4_0, p_4_2)),
 Not(And(p_4_1, p_4_2)),
 Or(p_4_0, p_4_1, p_4_2),
 Not(And(p_5_0, p_5_1)),
 Not(And(p_5_0, p_5_2)),
 Not(And(p_5_1, p_5_2)),
 Or(p_5_0, p_5_1, p_5_2),
 Not(And(p_6_0, p_6_1)),
 Not(And(p_6_0, p_6_2)),
 Not(And(p_6_1, p_6_2)),
 Or(p_6_0, p_6_1, p_6_2),
 Not(And(p_7_0, p_7_1)),
 Not(And(p_7_0, p_7_2)),
 Not(And(p_7_1, p_7_2)),
 Or(p_7_0, p_7_1, p_7_2),
 And,
 And,
 And,
 And,
 And,
 And(Or(Not(p_1_2))),
 And,
 And,
 And,
 And,
 And,
 And,
 And,
 And(Or(Not(p_4_1))),
 And(Or(Not(p_4_2))),
 And,
 And,
 And,
 And,
 And,
 And,
 And,
 And,
 And]

## Calculates the length of the path for each courier and sets the corresponding length variable to that value

In [103]:
if s.check() == sat:
    m = s.model()
    print(couriers_capacity)
    for c in range(number_of_couriers):
      for i in range(number_of_items):
        if m.evaluate(items_assigned[i][c]):
          print(f"item {i} is delivered by {c}, it has a size of: {items_size[i]}")
else:
    print("unsat")

[15, 7, 5]
item 0 is delivered by 0, it has a size of: 5
item 1 is delivered by 0, it has a size of: 6
item 2 is delivered by 0, it has a size of: 5
item 3 is delivered by 0, it has a size of: 3
item 4 is delivered by 0, it has a size of: 8
item 5 is delivered by 0, it has a size of: 1
item 7 is delivered by 0, it has a size of: 4
item 6 is delivered by 1, it has a size of: 1
